**Exemplo 02: Previsão de feedbacks de produtos B2W** 

 

Você recebeu um convite para uma consultoria, na qual deve desenvolver um modelo de previsões de feedbacks de clientes em produtos comprados na loja, que serão coletados do instagram. 

 

Os dados que você vai utilizar estão localizados em: 

https://raw.githubusercontent.com/americanas-tech/b2w-reviews01/refs/heads/main/B2W-Reviews01.csv 

 

Na coluna 'review_title' você vai encontrar feedbacks passados dos nossos clientes em nossos produtos e, na coluna 'overall_rating', a nota que foi dada. Esse é o único dado que temos para auxiliar na criação desse modelo de previsões. 


##### IMPORTAR AS BIBLIOTECAS

In [1]:
import pandas as pd
import numpy as np
# tokeniza
from tensorflow.keras.preprocessing.text import Tokenizer #type:ignore
# Ajusta o tamanho do vetor
from tensorflow.keras.preprocessing.sequence import pad_sequences  #type:ignore
# Define o modelo de rede neural utilizada
from tensorflow.keras.models import Sequential #type:ignore
# Camadas da rede neural
from tensorflow.keras.layers import Embedding, LSTM, Dense #type:ignore
# Otimizador de taxa de aprendizado
from tensorflow.keras.optimizers import Adam #type:ignore

from sklearn.model_selection import train_test_split

##### OBTER DADOS

In [10]:
try:

    ENDERECO_DADOS = 'https://raw.githubusercontent.com/americanas-tech/b2w-reviews01/refs/heads/main/B2W-Reviews01.csv'

    df = pd.read_csv(ENDERECO_DADOS, sep=',', encoding='utf-8')[['review_title', 'overall_rating']]
    #print(df.head(5))
                     
except Exception as e:
    print('ERRO AO OBTER DADOS', e)
 

       submission_date                                        reviewer_id  \
0  2018-01-01 00:11:28  d0fb1ca69422530334178f5c8624aa7a99da47907c44de...   
1  2018-01-01 00:13:48  014d6dc5a10aed1ff1e6f349fb2b059a2d3de511c7538a...   
2  2018-01-01 00:26:02  44f2c8edd93471926fff601274b8b2b5c4824e386ae4f2...   
3  2018-01-01 00:35:54  ce741665c1764ab2d77539e18d0e4f66dde6213c9f0863...   
4  2018-01-01 01:00:28  7d7b6b18dda804a897359276cef0ca252f9932bf4b5c8e...   

  product_id                                       product_name  \
0  132532965  Notebook Asus Vivobook Max X541NA-GO472T Intel...   
1   22562178               Copo Acrílico Com Canudo 500ml Rocie   
2  113022329  Panela de Pressão Elétrica Philips Walita Dail...   
3  113851581               Betoneira Columbus - Roma Brinquedos   
4  131788803  Smart TV LED 43" LG 43UJ6525 Ultra HD 4K com C...   

    product_brand      site_category_lv1       site_category_lv2  \
0             NaN            Informática                Notebook  

C:\Users\36131872024.1\AppData\Local\Temp\ipykernel_25428\2253079434.py:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ENDERECO_DADOS, sep=',', encoding='utf-8')#[['review_title', 'overall_rating']]


#### TRATAR DADOS

In [3]:
try:

    # Excluindo valores nao existente (nans)
    df = df.dropna(subset=['review_title', 'overall_rating'])

    #Transformando colunas em arrays
    texts = np.array(df['review_title'])
    rating = np.array(df['overall_rating'])

    print(df.head())

                     
except Exception as e:
    print('ERRO AO OBTER DADOS', e)
   

                       review_title  overall_rating
0                               Bom               4
1  Preço imbatível, ótima qualidade               4
2      ATENDE TODAS AS EXPECTATIVA.               4
3        presente mais que desejado               4
4            Sem duvidas, excelente               5


#### VETORIZAR

In [4]:
try:

    # Passo 1: tokenizar
    tokenizer = Tokenizer()

    # Passo 2: Criar o dicionário
    # fit_on_texts: Cria o vacabulário, através do dicionário
    # associando cada token a um indice numérico
    # lembrando que se a palavra aparecer mais de uma vez, ela vai receber o mesmo indice numerico
    tokenizer.fit_on_texts(texts)

    # Passo3: Vetorizar, ou seja, transformar os tokens em números a partir do dicionario criado no passo 2
    vetores = tokenizer.texts_to_sequences(texts)

    # Passo 4: Padronização do tamanho do vetor - pad
    padded_vetores = pad_sequences(vetores) 

    print(padded_vetores)
    
                     
except Exception as e:
    print('ERRO AO VETORIZAR TEXTOS', e)
    exit()

[[   0    0    0 ...    0    0    3]
 [   0    0    0 ... 2620   30   16]
 [   0    0    0 ...  349   45  155]
 ...
 [   0    0    0 ...    0    9    1]
 [   0    0    0 ...    4   19    3]
 [   0    0    0 ...    1    4   51]]


##### CONSTRUIR A REDE NEURAL

In [5]:
try:

    # Constantes do modelo

    # 1ª Constante: Tamanho do vacabulário (tamanho do dicionario do modelo de contexto)
    VOCAB_SIZE = len(tokenizer.word_index) + 1

    # 2ª Constante: Tamanho máximo da sequencia
    # É o comprimento máximo de um texto
    MAX_SEQUENCE_LENGTH = padded_vetores.shape[1] # linha 0 coluna 1 a quantidade de colunas vai mostrar o comprimento maximo do vetor (a qtde de palavras)

    # 3ª Constante: Tamanho do vetor de entrada
    # A literatura recomenda que seja iniciado pela quantidade igual a raiz quadrada do tamanho do vocabulário
    # Se o volume de dados for de larga escala, pode-se testar iniciando com um tamanho maior
    # Se o volume de dados for muito pequeno, pode-se testar iniciando com um tamanho menor
    # Cuidado com o overfitting, que é quando o modelo aprende demais e começa a perder a capacidade de generalizar com novos dados
    # Não consegue observar todas as diferenças textuais
    # Overfitting pode ser observado no treino da rede neural
    VETOR_LENGTH = int(np.sqrt(VOCAB_SIZE))

    # Inicia-se a construção da rede neural
    # Sequential é fluxo linear de camadas (conforme visto na Aula02_RNA.pptx)
    # São processadas em ordem
    model = Sequential()

    # Camada de entrada
    # Embedding, na qual os vetores de texto são inseridos
    model.add(Embedding(input_dim=VOCAB_SIZE,
                        output_dim =VETOR_LENGTH, # Output da camada de entrada. input pra camada oculta
                        input_length=MAX_SEQUENCE_LENGTH))
    
    # Camada oculta ou intermediária
    # LSTM - long short term memory, em portugues " memoria de curto e longo prazo"
    # É onde a magia acontece. É onde o modelo treina baseado nos seus vetores
    # Números de unidades de memória, que é a quantidade de neuronios. Quanto mais neuronios, maior a acurácia
    # No primeiro TESTE experimente somente com uma camada! Cuidado com o overfitting!
    # Se for necessário adicionar mais camadas, basta repetir o comando abaixo
    
    # Primeira camada oculta
    model.add(LSTM(128)) # Uma camada com 128 neuronios

    # Se necessário adicionar outra camada oculta, repita model.add(LSTM(qtde de neuronios))

    # Camada de saída - Camada densa
    # Regressão que é ocaso desse exemplo. Somente 1 camada
    model.add(Dense(1))
    # Construir o modelo
    # É literalmente pegar as definições anteriores e construir o modelo
    # input_shape: é o formato dos dados de entrada e ainda o tamanho máximo do texto (MAX_SEQUENCE_LENGHT)

    model.build(input_shape=(None, MAX_SEQUENCE_LENGTH))

    # Otimizador de taxa de aprendizado. 
    # Importante para ajustar em casos de overfitting
    # Adam é o otimizador que ajusta essa taxa de aprendizado
    # parametro learn_rating: Quanto menor, melhor o aprendizado. Menos risco de overfitting
    otimizador = Adam(learning_rate=0.0005)

    # Compilar o modelo
    # Verificar se há ou não algum erro
    # É informado o otimizador e a métrica de perda (LOSS)
    # loss - erro quadro médio (mean_squared_error)
    model.compile(optimizer=otimizador, loss='mean_squared_error')

    model.summary()
    print('Modelo configurado e criado')


except Exception as e:
    print('ERRO AO CONSTRUIR A REDE NEURAL', e)

C:\Users\36131872024.1\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 31, 124)        │     1,936,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       129,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,066,545 (7.88 MB)

 Trainable params: 2,066,545 (7.88 MB)

 Non-trainable params: 0 (0.00 B)

Modelo configurado e criado


##### TREINAR O MODELO

In [6]:
try:

    X_train, X_test, y_train, y_test = train_test_split(
        padded_vetores,
        rating,
        test_size=0.2,
        random_state=42
    )

    # O treino da rede neural
    model.fit(
        X_train,
        y_train,
        epochs=5,
        batch_size=128,
        validation_data=(X_test, y_test))
    
except Exception as e:
    print('ERRO AO TREINAR O MODELO', e)

Epoch 1/5
826/826 ━━━━━━━━━━━━━━━━━━━━ 30s 35ms/step - loss: 1.9668 - val_loss: 0.7093
Epoch 2/5
826/826 ━━━━━━━━━━━━━━━━━━━━ 29s 35ms/step - loss: 0.6182 - val_loss: 0.6730
Epoch 3/5
826/826 ━━━━━━━━━━━━━━━━━━━━ 29s 35ms/step - loss: 0.5504 - val_loss: 0.6590
Epoch 4/5
826/826 ━━━━━━━━━━━━━━━━━━━━ 29s 35ms/step - loss: 0.5019 - val_loss: 0.6503
Epoch 5/5
826/826 ━━━━━━━━━━━━━━━━━━━━ 28s 34ms/step - loss: 0.4689 - val_loss: 0.6498


##### TESTAR O MODELO

In [8]:
try:

    novos_textos = [
        'Muito bom, gostei bastante. Top demais! Compensa muito!',
        "Não recomendo, péssimo produto. Não funciona",
        "Muito bom, Americanas. Só faz merda"
    ]

    novas_sequencias = tokenizer.texts_to_sequences(novos_textos)
    novas_sequencias_padded = pad_sequences(novas_sequencias)

    predicoes = model.predict(novas_sequencias_padded)
    print("Previsões:", predicoes)
    
except Exception as e:
    print('ERRO AO TESTAR O MODELO', e)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
Previsões: [[3.0520608 ]
 [0.13965307]
 [3.8089793 ]]
